In [1]:
import numpy as np
import CHONK_XL as chxl
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import xsimlab as xs
import CHONK_cpp as ch
import zarr
import helplotlib as hpl
import helper
%matplotlib widget
%load_ext xsimlab.ipython

IndentationError: expected an indented block (environment.py, line 60)

In [2]:


@xs.process
class CustomParameters:
    label_array = xs.variable(intent = 'out', dims = (('y','x'), ('node')))
    label_list = xs.any_object()
    CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
    nx = xs.foreign(chxl.ChonkBase, "nx")
    ny = xs.foreign(chxl.ChonkBase, "ny")
    dx = xs.foreign(chxl.ChonkBase, "dx")
    dy = xs.foreign(chxl.ChonkBase, "dy")
    
    active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
#     landscape = xs.any_object()

    def initialize(self):

        self.label_list = []
    
        self.label_array = np.zeros((nx*ny), dtype = np.int)
        
        # Labelling the second half of the landscape as label 1
        self.label_array[round((nx*ny)/2):] = 1

        self.label_list.append(ch.label(0))
        self.label_list[-1].m = 0.45;
        self.label_list[-1].n = 1;
        self.label_list[-1].base_K = 1e-5;
        self.label_list[-1].Ks_modifyer = 1.2;
        self.label_list[-1].Kr_modifyer = 0.8;
        self.label_list[-1].dimless_roughness = 0.5;
        self.label_list[-1].V = 0.5;
        self.label_list[-1].dstar = 1;
        self.label_list[-1].threshold_incision = 0;
        self.label_list[-1].threshold_entrainment = 0;
        self.label_list[-1].kappa_base = 1e-4;
        self.label_list[-1].kappa_r_mod = 0.8;
        self.label_list[-1].kappa_s_mod = 1.2;
        self.label_list[-1].critical_slope = 0.57835;
        self.label_list[-1].sensitivity_tool_effect = 1;

        self.label_list.append(ch.label(1))
        self.label_list[-1].m = 0.45;
        self.label_list[-1].n = 1;
        self.label_list[-1].base_K = 1e-5;
        self.label_list[-1].Ks_modifyer = 1;
        self.label_list[-1].Kr_modifyer = 0.3;
        self.label_list[-1].dimless_roughness = 0.5;
        self.label_list[-1].V = 0.1;
        self.label_list[-1].dstar = 1;
        self.label_list[-1].threshold_incision = 0;
        self.label_list[-1].threshold_entrainment = 0;
        self.label_list[-1].kappa_base = 1e-4;
        self.label_list[-1].kappa_r_mod = 0.8;
        self.label_list[-1].kappa_s_mod = 1.2;
        self.label_list[-1].critical_slope = 0.57835;
        self.label_list[-1].sensitivity_tool_effect = 1;

        self.CHONK.initialise_label_list(self.label_list)
        self.CHONK.update_label_array(self.label_array.ravel())
        
        
@xs.process
class UpliftLandscape(chxl.Uplift):
	uplift_done = xs.variable(intent = "out")
	runner_done = xs.foreign(chxl.Runner, "runner_done")
	uplift = xs.variable(intent = 'out', dims = [('y','x'), ('node')])
	switch_time = xs.variable(intent = 'in')
	CHONK = xs.foreign(chxl.ChonkBase, "CHONK")
	active_nodes = xs.foreign(chxl.ChonkBase, "active_nodes")
	nx = xs.foreign(chxl.ChonkBase, "nx")
	ny = xs.foreign(chxl.ChonkBase, "ny")
	dx = xs.foreign(chxl.ChonkBase, "dx") 
	dy = xs.foreign(chxl.ChonkBase, "dy")
    
	rate = xs.variable()
	minrate = xs.variable(default=0)
	idx_N1 = xs.variable()
	idx_N2 = xs.variable()
	idx_N3 = xs.variable()
	idx_N4 = xs.variable()
    

	def initialize(self):
		self.uplift = np.zeros((ny,nx))
		self.uplift[self.idx_N1:self.idx_N2,:] = 1
		self.uplift[self.idx_N3:self.idx_N4,:] = 1
		self.uplift[self.idx_N2:self.idx_N3,:] = self.minrate
		from scipy.ndimage import gaussian_filter
		self.uplift = gaussian_filter(self.uplift, sigma=2)
		self.uplift /= self.uplift.max()
		self.uplift *= self.rate
		self.uplift[[-1,0],:] = 0
		self.done = False
        

	@xs.runtime(args=['step_delta','step_end'])
	def run_step(self, dt, timing):
		self.CHONK.add_external_to_surface_elevation_tp1(self.uplift.ravel() * dt)
# 		print(self.CHONK.get_neighbours_for_debugging(99,50))
		self.uplift_done = True
		
# landscape.uplift4StSt

In [3]:
model = xs.Model({"ChonkBase": chxl.ChonkBase,
                "Runner": chxl.Runner,
                "Topography": chxl.Topography,
#                 "Uplift": UpliftLandscapeStSt,
                "Uplift": UpliftLandscape,
#                 "Uplift": UpliftLandscapeNF1,
#                 "Uplift": chxl.Uplift,
                "Lake": chxl.Lake,
                "Precipitation": chxl.Precipitation,
#                 "DefaultParameters": chxl.DefaultParameters,
                "DefaultParameters": CustomParameters,
                "Flow": chxl.Flow,
                "Fluvial": chxl.Fluvial,
                "Hillslope": chxl.Hillslope
            })



In [4]:
ny,nx = 100,100
dy,dx = 200,200

In [7]:
# %create_setup model
import xsimlab as xs
time = np.arange(0,1e7,1000)
otime = time[::20]

# init_z = np.load("./initial_topo_100_100.npy")
init_z = np.random.rand(ny,nx)
rate = 0.5e-3
idx_N1 = 10
idx_N2 = 30
idx_N3 = 70
idx_N4 = 80
# init_z[0,:] = 1000
# init_z[0,:] = 0
U = np.zeros((ny,nx)) + 2e-4

Utime = 1e8
ds_in = xs.create_setup(
    model=model,
    clocks={
        'time': time,
        'otime': otime
    },
    master_clock='time',
    input_vars={
        'ChonkBase__dx': dx,
        'ChonkBase__dy': dy,
        'ChonkBase__nx': nx,
        'ChonkBase__ny': ny,
        'ChonkBase__depths_res_sed_proportions': 10,
        'ChonkBase__n_depth_sed_tracking': 50,
        'ChonkBase__boundary_conditions': "periodic_EW",
        'Topography__initial_elevation': init_z,
        'Uplift__rate': rate,
        'Uplift__idx_N1': idx_N1,
        'Uplift__idx_N2': idx_N2,
        'Uplift__idx_N3': idx_N3,
        'Uplift__idx_N4': idx_N4,
        'Uplift__switch_time': Utime,
#         'Lake__method': ('batch', ['implicit', 'explicit']),
        'Lake__method': 'explicit',
        'Lake__evaporation': True,
        'Lake__evaporation_rate':  1.3,#('batch', np.arange(1,1.6,0.1)),
        'Flow__threshold_single_flow': 1e32,
        'Precipitation__precipitation_rate': 1,#('batch', np.arange(0.3,2,0.2)), #1
    },
    output_vars={
        'Topography__topography': 'otime',
        'Topography__sed_height': 'otime',
        'Flow__Qw': 'otime',
        'Flow__water_balance_checker': 'time',
        'Lake__lake_depth': 'otime',
        'Lake__topolake': 'otime',
        'Fluvial__Qs': 'otime',
        'Hillslope__Qs': 'otime',
        'ChonkBase__Q_sout_lab_N': 'otime',
        'ChonkBase__Q_sout_lab_S': 'otime',
        'ChonkBase___stratigraphator': 'otime',
    
    }
)


KeyError: "ChonkBase___stratigraphator is/are not valid key(s) for variables in model <xsimlab.Model (10 processes, 21 inputs)>\nChonkBase\n    dx                             [in] Spacing in X direction\n    dy                             [in] Spacing in Y direction\n    nx                             [in] Number of nodes in X direction\n    ny                             [in] Number of nodes in Y direction\n    boundary_conditions            [in]\n    depths_res_sed_proportions  [inout] Depth resolution for saving...\n    n_depth_sed_tracking           [in]\nRunner\nUplift\n    switch_time                    [in]\n    rate                           [in]\n    minrate                        [in]\n    idx_N1                         [in]\n    idx_N2                         [in]\n    idx_N3                         [in]\n    idx_N4                         [in]\nTopography\n    initial_elevation              [in] ('y', 'x') or ('node',) \n    initial_carving                [in]\nLake\n    me

In [9]:
from dask.distributed import Client
if('client' in locals()):
    if(isinstance(client,Client)):
        client.close()
client = Client(n_workers=6, threads_per_worker=1)  


zg = zarr.group("rift_lake_evaporation_2.zarr", overwrite=True)
with model,xs.monitoring.ProgressBar():
    out_ds = ds_in.xsimlab.run(store = zg, batch_dim='batch', parallel=True, scheduler=client)
#     out_ds = ds_in.xsimlab.run(store = zg)
#     out_ds = mod1.xsimlab.run()  
out_ds.x.values[0] = 0
out_ds.y.values[0] = 0
# out_ds["topolake"] = out_ds["Topography__topography"] + out_ds["Lake__lake_depth"]

             0% | initialize /var/folders/t0/7zf23f454vn3_kkgrbf4c1500000gp/T/ipykernel_21417/980392377.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
             0% | run step 0/9999 /var/folders/t0/7zf23f454vn3_kkgrbf4c1500000gp/T/ipykernel_21417/980392377.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you 

1.0114|178.559|176.546|0.955076|value:-45.1858
1.00081|138.774|138.662|0.925706|value:-37.6088
1.00257|91.8739|91.6383|0.939938|value:-23.618
1.02303|181.483|177.398|0.89624|value:-39.5322
1.01062|93.8791|92.8924|0.89624|value:-25.3394
1.00141|747.731|746.681|0.955106|value:-60.107
1.00401|356.903|355.478|0.983175|value:-86.1544


             0% | run step 1/9999 

1.00048|371.943|371.765|0.982057|value:-83.9821
1.00364|202.718|201.983|0.993521|value:-116.142
1.00611|219.871|218.535|0.961711|value:-50.2076
1.01257|367.241|362.681|0.854386|value:-31.4828
1.03276|179.2|173.516|0.871799|value:-44.5027
1.02174|96.5626|94.5077|0.871799|value:-28.8447
1.00071|780.558|780.007|0.956093|value:-88.5206
1.00094|148.477|148.337|0.986079|value:-18.1969
1.04855|106.689|101.748|0.850441|value:-36.9884
1.00214|461.183|460.196|0.941824|value:-88.5249
1.00089|1487|1485.68|0.911781|value:-53.319
1.00144|1809.87|1807.28|0.911781|value:-78.7509
1.0165|173.297|170.484|0.956784|value:-85.9202
1.00575|100.189|99.617|0.919327|value:-50.1763
1.00177|295.701|295.179|0.939149|value:-38.7698
1.00048|371.942|371.765|0.982057|value:-83.9821
1.00364|202.718|201.983|0.993521|value:-116.142
1.00611|219.871|218.535|0.961711|value:-50.2076
1.01248|370.042|365.482|0.854386|value:-31.4828
1.03276|179.2|173.516|0.871799|value:-44.5027
1.02174|96.5626|94.5076|0.871799|value:-28.8447
1.

             0% | run step 2/9999 

...

███████▊    78% | run step 7752/9999 

1|4.22191e+06|4.22191e+06|0.994548|value:-459.982


███████▉    79% | run step 7934/9999 

1.00071|1829.85|1828.56|0.999401|value:-2285.12


██████████ 100% | Simulation finished in 24:15 

In [10]:
from ipyfastscape import TopoViz3d



# app = TopoViz3d(out_ds, canvas_height=600, time_dim="otime", elevation_var = "Topography__topography" )
app = TopoViz3d(out_ds, canvas_height=600, time_dim="otime", elevation_var = "Lake__topolake" )

app.show()

Output(layout=Layout(height='640px'))

In [ ]:
#np.save("initial_topo_100_100.npy", out_ds.Topography__topography.values[-1])
# np.save("sed4testHS_CHONK.npy", out_ds.Topography__sed_height.sel({'otime':1.001e6}, method="nearest").values)
# np.save("sed4testtopo_CHONK.npy", out_ds.Topography__topography.sel({'otime':1.001e6}, method="nearest").values)

In [ ]:
fig,ax = plt.subplots()
ax.plot(out_ds.time.values, out_ds.sel({'batch':1}).Flow__water_balance_checker.values)
# ax.plot(out_ds.time.values, np.abs(out_ds.Flow__water_balance_checker.values))
ax.set_yscale('log')

In [ ]:
fig, ax = plt.subplots()
cb = ax.imshow(np.log10(out_ds["Flow__Qw"].sel({'batch':1, 'otime':9900000}).values), cmap = "Blues")
plt.colorbar(cb)

In [12]:
import matplotlib.gridspec as gridspec
import xarray as xr
import matplotlib
%matplotlib widget

font = {'family' : 'Helvetica Neue',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

out_ds = xr.open_zarr("rift_lake_evaporation_2.zarr/")
out_ds.x.values[0] = 0
out_ds.y.values[0] = 0

tout_ds = out_ds.sel({"batch":3})
# tout_ds = xr.open_zarr("rift_lake_tracking.zarr")

zumzumN = tout_ds.ChonkBase__Q_sout_lab_N.sum(dim = ['x'])
zumzumS = tout_ds.ChonkBase__Q_sout_lab_S.sum(dim = ['x'])

A = zumzumN.sel({'n_labels':0})
B = A + zumzumN.sel({'n_labels':1})
C = zumzumS.sel({'n_labels':0})
D = C + zumzumS.sel({'n_labels':1})

plt.close('all')


fig =  plt.figure(constrained_layout=False, figsize = (12,8))

grid = gridspec.GridSpec(100, 1, wspace=0, hspace=0)

ax1,ax2 = fig.add_subplot(grid[0:80, 0]),fig.add_subplot(grid[80:, 0])

ax1.fill_between(tout_ds.otime, 0,A, lw = 0, color = '#bdbdbd', label = "From the Southern half")
ax1.fill_between(tout_ds.otime, A, B, lw = 0, color = '#474747', label = "From the Northern Half")

ax2.fill_between(tout_ds.otime, 0,C, lw = 0, color = '#bdbdbd', label = "S label 0")
ax2.fill_between(tout_ds.otime, C,D, lw = 0, color = '#474747', label = "S label 1")

ax1.legend(loc = 'upper left')

whylim = [0,150000]
whylim2 = [0,150000/0.8 * 0.2]
ax2.set_ylim(whylim2)
ax1.set_ylim(whylim)
ax2.invert_yaxis()

ax1.get_yaxis().set_label_coords(-0.1,0.55)
ax2.get_yaxis().set_label_coords(-0.1,0.7)

ax1.grid(ls = '--', color = 'k', lw = 1, alpha = 0.6) 
ax2.grid(ls = '--', color = 'k', lw = 1, alpha = 0.6) 

ax2.set_xlabel("Time (yrs)")
ax2.set_ylabel(r"$Q_s$ [out to the S] ($m^3/yrs$)")
ax1.set_ylabel(r"$Q_s$ [out to the N] ($m^3/yrs$)")
# ax.plot(out_ds.otime, zumzumN, lw = 1, color = 'k')
# ax.plot(out_ds.otime, zumzumS, lw = 1, color = 'r')
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/var/folders/t0/7zf23f454vn3_kkgrbf4c1500000gp/T/ipykernel_21417/2536336896.py:61: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  plt.tight_layout()


In [13]:
from importlib import reload  
import plotter as pol
import warnings;warnings.simplefilter('ignore')
reload(pol)

pol.anim_lake_cross_section(
	out_ds, # The input ds 
	fname = "rift_lake_explicit_evaporation_batch3",
	timedim = "otime", # the time dimension
	batch_dim = 'batch',
    batch_val = 3,
	cross_section_dir = 'x', # is the cross section in x or y direction
	xy_cross_section = 10000, # coordinate on the other axis
	color_bedrock = 'gray', # color of the bedrock
	color_sediments = 'orange', # color of the bedrock
	color_water = 'blue', # color of the water
	z_min = 0, # minimum z on the cross_section, if left to none -> min of all
	z_max = 600, # max z on the cross_section, if left to none -> min of all
	# Map parameters
	cmap_elev = 'gist_earth', # cmap of the cross-section
	alpha_hillshade = 0.5, # transparency of the hillshade
    czmin = 0,
    czmax = 600,
    figsize = (12,4.5),
    custom_tickszz = np.arange(0,21000,5000, dtype = np.int),
)

  0%|          | 0/500 [00:00<?, ?it/s]

distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

Movie created at rift_lake_explicit_evaporation_batch3.mp4
GIF created at rift_lake_explicit_evaporation_batch3.gif


In [17]:
arr = np.random.rand(2,76)

In [18]:
arr.shape

(2, 76)

In [19]:
arr = np.array([[0,0], [1,-1], [2,-2]])

In [31]:
arr.ravel()[np.where(arr.ravel() >= 0 )]

array([0, 0, 1, 2])

In [61]:
arr = np.vstack([np.arange(500), np.arange(500)])
arr.shape

(2, 500)

In [62]:
arr[:, ]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [64]:
np.logical_and(arr[1,:] < 200 , arr[1,:] > 150)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,